# Imports

In [28]:
import pandas as pd
from datetime import datetime as dt
from pathlib import Path
from plotly import graph_objects as go

# Settings

In [29]:
inputs_dir = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\13 Nachrüstung RÜ\Datenauswertung\Monatsdaten")

save_dir = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\13 Nachrüstung RÜ\Datenauswertung\Plots")

idx_niveaus = [0,3,6,9]

prop_dict = {
    'Niveau HE Schulstrasse [cm]': {
        'datum_ok': dt(2023, 3, 1, 16),
        'h_entlastung': 121
    },

    'Niveau HE Löchliweg [cm]': {
        'datum_ok': dt(2000, 1, 1),
        'h_entlastung': 37
    },

    'Niveau HE Bodenstrasse [cm]': {
        'datum_ok': dt(2023, 3, 23, 9, 30),
        'h_entlastung': 30
    },

    'Niveau HE Mülenen [cm]': {
        'datum_ok': dt(2023, 3, 23, 16),
        'h_entlastung': 0
    }
}

# Read data

In [30]:
files_list = list(inputs_dir.glob('*.csv'))

In [31]:
datumstr_list = sorted([x.stem.split('_')[-1] for x in files_list])
datum1 = datumstr_list[0]
datum2 = datumstr_list[-1]

In [32]:
date_format = '%d.%m.%Y %H:%M'
data = []
for file in files_list:
    df = pd.read_csv(file, sep=';', header='infer', index_col=0, parse_dates=[0],
        date_format=date_format)
    cols_dict = {}
    for col_name_old in df.columns:
        col_name_new = col_name_old.replace('\t', '; ')
        cols_dict.update({col_name_old: col_name_new})
    df.rename(columns=cols_dict, inplace=True)

    data.append(df)
data = pd.concat(data)
data.sort_index(inplace=True)

# Plots

## Niveaus

In [34]:
niveaus = data.iloc[:,idx_niveaus]
fig = go.Figure()
for name in niveaus:
    fig.add_trace(go.Scatter(x=niveaus.index, y=niveaus[name], name=name))

fig.update_xaxes(title='Zeit')
fig.update_yaxes(title='Niveau [cm]')

save_path = save_dir / f'Niveaus_alle_{datum1}-{datum2}.html'
fig.write_html(save_path)

## Anzahl Entlastungen

In [35]:
n_entlastungen = data.iloc[:,[1,4,7,10,12]]
fig = go.Figure()
for name in n_entlastungen:
    fig.add_trace(go.Scatter(x=n_entlastungen.index, y=n_entlastungen[name], name=name))

fig.update_xaxes(title='Zeit')
fig.update_yaxes(title='Anzahl Entlastungen')

save_path = save_dir / f'Anzahl-Entlastungen_alle_{datum1}-{datum2}.html'
fig.write_html(save_path)